In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import numpy as np
import json
import pickle

import pandas as pd
from IPython.display import display
pd.options.display.max_rows = None
pd.options.display.max_columns = None

[nltk_data] Downloading package punkt to /home/manish/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/manish/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
words = []
classes = []
document = []
ignore = ['?', '!']
data = open('intents.json').read()
intents = json.loads(data)

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        document.append((word, intent['tag']))
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [5]:
training = []
output = [0] * len(classes)

for doc in document:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    for w in words:
        bag.append(int(1)) if w in pattern_words else bag.append(int(0))
    
    output_row = list(output)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
    
np.random.shuffle(training)
training = np.array(training)

x_train = training[:, 0]
y_train = training[:, 1]


/home/manish/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [6]:
model = keras.Sequential()
model.add(Dense(128, input_shape = (len(x_train[0]),), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation = 'softmax'))

sgd = SGD(decay = 1e-6, learning_rate = 0.01, momentum = 0.9, nesterov = True)

model.compile(
    loss = 'categorical_crossentropy',
    optimizer = sgd,
    metrics = 'accuracy'
)

history = model.fit(tf.data.Dataset.from_tensor_slices((x_train, y_train)),
                   epochs = 10,
                   batch_size = 5,
                   verbose = 1)

model.save('model.h5', history)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).